![example](images/director_shot.jpeg)

# Project Title

**Authors:** Student 1, Student 2, Student 3
***

## Overview

A one-paragraph overview of the project, including the business problem, data, methods, results and recommendations.

## Business Problem

Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.

***
Questions to consider:
* What are the business's pain points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?
***

## Data Understanding

Describe the data being used for this project.
***
Questions to consider:
* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?
***

In [33]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile

%matplotlib inline

In [34]:
# Here you run your code to explore the data
dfbom = pd.read_csv('data/zippeddata/bom.movie_gross.csv')
# df.sort_values('year')

df_imdb_name = pd.read_csv('data/zippeddata/imdb.name.basics.csv')
# df.info()
# df.head()

df_imdb_titleb = pd.read_csv('data/zippeddata/imdb.title.basics.csv')
# df.info()
# df.head()

# df = pd.read_csv('data/zippeddata/bom.movie_gross.csv')
# df.info()
# df.head()

df_imdb_title = pd.read_csv('data/zippeddata/imdb.title.crew.csv')
# df.info()
# df.head()


df_imdb_principals = pd.read_csv('data/zippeddata/imdb.title.principals.csv')
# df.info()
# df.head()

df_imdb_ratings = pd.read_csv('data/zippeddata/imdb.title.ratings.csv')
# df.info()
# df.head()

tsv_file='data/zippeddata/rt.movie_info.tsv'
csv_table=pd.read_table(tsv_file,sep='\t')
csv_table.to_csv('rt.movie_info.csv',index=False)

# csv_table.info()
# csv_table.head()


tsv_file='data/zippeddata/rt.reviews.tsv'
csv_table=pd.read_table(tsv_file,sep='\t')
csv_table.to_csv('rt.reviews.csv',index=False)

# csv_table.info()
# csv_table.head()

df_tmdb_movies = pd.read_csv('data/zippeddata/tmdb.movies.csv')
# df.info()
# df.head()

df_budget = pd.read_csv('data/zippeddata/tn.movie_budgets.csv')





In [35]:
#joining budget to imdb table



## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

In [61]:
# Here you run your code to clean the data
# Here you run your code to model the data

# cleaned the first data set
dfbom['domestic_gross'].dropna(inplace= True)
dfbom['domestic_gross'] = pd.to_numeric(dfbom['domestic_gross'], errors ='coerce')
dfbom['foreign_gross'].dropna(inplace= True)
dfbom['foreign_gross'] = pd.to_numeric(dfbom['foreign_gross'], errors ="coerce")

dfbom.head()
dfbom['total_gross']= dfbom['domestic_gross']+ dfbom['foreign_gross']

dfbom.head()

# df_combined_budget = dfbom.merge(df_budget, left_on = "title", right_on = "movie")




df_budget[df_budget.columns[3:6]] = df_budget[df_budget.columns[3:6]].replace('[\$,]', '', regex=True).astype(float)

df_budget.info()

# created another row to determine wether the film grossed a profit
df_budget['profit'] = df_budget['worldwide_gross']- df_budget['production_budget']
df_budget['fin_success'] =  (df_budget['profit'] > 0 )

df_budget.head()
df_budget['fin_success'].value_counts() # 3657 made money, 2125 lost money
df_budget['fin_success'].describe()     

df_imdb_comb = pd.merge(df_imdb_titleb,df_imdb_ratings,on='tconst')

df_budget_genre = df_budget.merge(df_imdb_titleb, left_on='movie', right_on='primary_title')

df_budget_genre.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5782 non-null   int64  
 1   release_date       5782 non-null   object 
 2   movie              5782 non-null   object 
 3   production_budget  5782 non-null   float64
 4   domestic_gross     5782 non-null   float64
 5   worldwide_gross    5782 non-null   float64
 6   profit             5782 non-null   float64
 7   fin_success        5782 non-null   bool   
dtypes: bool(1), float64(4), int64(1), object(2)
memory usage: 322.0+ KB


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,profit,fin_success,tconst,primary_title,original_title,start_year,runtime_minutes,genres
3810,68,"Jul 6, 2001",Cure,10000.0,94596.0,94596.0,84596.0,True,tt5936960,Cure,Cure,2014,NaN,NaN
3811,70,"Apr 1, 1996",Bang,10000.0,527.0,527.0,-9473.0,False,tt6616538,Bang,Bang,2015,NaN,NaN
3812,73,"Jan 13, 2012",Newlyweds,9000.0,4584.0,4584.0,-4416.0,False,tt1880418,Newlyweds,Newlyweds,2011,95.0,"Comedy,Drama"
3813,78,"Dec 31, 2018",Red 11,7000.0,0.0,0.0,-7000.0,False,tt7837402,Red 11,Red 11,2019,77.0,"Horror,Sci-Fi,Thriller"
3814,81,"Sep 29, 2015",A Plague So Pleasant,1400.0,0.0,0.0,-1400.0,False,tt2107644,A Plague So Pleasant,A Plague So Pleasant,2013,76.0,"Drama,Horror,Thriller"


In [67]:
def splitDataFrameList(df,target_column,separator):
    row_accumulator = []
    def splitListToRows(row, separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    df.apply(splitListToRows, axis=1, args = (separator, ))
    new_df = pd.DataFrame(row_accumulator)
    return new_df
df_imdb_mtitle = df_imdb_titleb[df_imdb_titleb['genres'].notnull()]
single_genre = splitDataFrameList(df_imdb_mtitle,'genres',',')

single_genre.groupby('genres')['genres'].value_counts()

genres       genres     
Action       Action         10335
Adult        Adult             25
Adventure    Adventure       6465
Animation    Animation       2799
Biography    Biography       8722
Comedy       Comedy         25312
Crime        Crime           6753
Documentary  Documentary    51640
Drama        Drama          49883
Family       Family          6227
Fantasy      Fantasy         3516
Game-Show    Game-Show          4
History      History         6225
Horror       Horror         10805
Music        Music           4314
Musical      Musical         1430
Mystery      Mystery         4659
News         News            1551
Reality-TV   Reality-TV        98
Romance      Romance         9372
Sci-Fi       Sci-Fi          3365
Short        Short             11
Sport        Sport           2234
Talk-Show    Talk-Show         50
Thriller     Thriller       11883
War          War             1405
Western      Western          467
Name: genres, dtype: int64

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

In [27]:
# Here you run your code to model the data




## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***